# Random Forest

## Prior Progress
Random Forest 13 did shuffle and cross validation for 70% accuracy.

## Next Task
Bring in RBC data. Must go back to Random Forest 08 for code that merged nucleus with RBC.

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2022-06-06 16:49:28.418002
Python 3.8.10
sklearn 1.0.2


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.feature_selection import RFE
import joblib # used to dump/load sklearn models

In [3]:
BASE_DIR='/home/jrm/ShepherdML/TumorII/'
NUCLEUS_FILES=[
    'Nucleus_Rollup_0.csv',
    'Nucleus_Rollup_1.csv',
    'Nucleus_Rollup_2.csv',
    'Nucleus_Rollup_3.csv',
    'Nucleus_Rollup_4.csv',
    'Nucleus_Rollup_5.csv']
RBC_FILES=[
    'RBC_Rollup_0.csv',
    'RBC_Rollup_1.csv',
    'RBC_Rollup_2.csv',
    'RBC_Rollup_3.csv',
    'RBC_Rollup_4.csv',
    'RBC_Rollup_5.csv']
CLASSES=range(1,6)   # we only have 1 thru 5 right now
MODELS_DIR='/home/jrm/Adjeroh/Naved/models/RandomForest.10/'

In [4]:
print(datetime.datetime.now())
PREFIX='abc_'
def set_prefix(prefix):
    global PREFIX
    PREFIX=prefix
def get_prefix(col_name):
    global PREFIX
    return PREFIX+col_name  # hard coded for now
def load_all_classes():
    X = None
    y = None
    for i in CLASSES:
        print(datetime.datetime.now())
        print("Loading class",i)
        Xnuc = pd.read_csv(BASE_DIR+NUCLEUS_FILES[i])
        Xnuc.set_index(['PatchNumber'],inplace=True)
        set_prefix('Nuc_')
        Xnuc.rename(get_prefix,axis='columns',inplace=True)
        Xrbc = pd.read_csv(BASE_DIR+RBC_FILES[i])
        Xrbc.set_index(['PatchNumber'],inplace=True)
        set_prefix('Rbc_')
        Xrbc.rename(get_prefix,axis='columns',inplace=True)
        Xi = Xnuc.join(Xrbc,how='outer')
        #Xi.drop(columns=['PatchNumber'],inplace=True)
        size = len(Xi)
        yi = np.ones(size) * i   # e.g. class 3
        if X is None:
            X = Xi
            y = yi
        else:
            X = pd.concat( (X,Xi) )
            y = np.concatenate( (y,yi) )
    X.fillna(0,inplace=True)  
    return X,y
Xtrain,ytrain=load_all_classes()
Xtrain

2022-06-06 16:49:30.217744
2022-06-06 16:49:30.217989
Loading class 1
2022-06-06 16:49:37.080161
Loading class 2
2022-06-06 16:49:44.909911
Loading class 3
2022-06-06 16:49:48.926818
Loading class 4
2022-06-06 16:49:51.076364
Loading class 5


,Nuc_ObjectNumber_count,Nuc_ObjectNumber_mean,Nuc_ObjectNumber_std,Nuc_ObjectNumber_min,Nuc_ObjectNumber_25%,Nuc_ObjectNumber_50%,Nuc_ObjectNumber_75%,Nuc_ObjectNumber_max,Nuc_AreaShape_Area_count,Nuc_AreaShape_Area_mean,...,Rbc_Number_Object_Number_75%,Rbc_Number_Object_Number_max,Rbc_Parent_ShrinkRBC_count,Rbc_Parent_ShrinkRBC_mean,Rbc_Parent_ShrinkRBC_std,Rbc_Parent_ShrinkRBC_min,Rbc_Parent_ShrinkRBC_25%,Rbc_Parent_ShrinkRBC_50%,Rbc_Parent_ShrinkRBC_75%,Rbc_Parent_ShrinkRBC_max
PatchNumber,,,,,,,,,,,,,,,,,,,,,
1209,23.0,12.0,6.782330,1.0,6.50,12.0,17.50,23.0,23.0,465.391304,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1210,15.0,8.0,4.472136,1.0,4.50,8.0,11.50,15.0,15.0,375.933333,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1211,40.0,20.5,11.690452,1.0,10.75,20.5,30.25,40.0,40.0,526.725000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1212,15.0,8.0,4.472136,1.0,4.50,8.0,11.50,15.0,15.0,451.133333,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1213,18.0,9.5,5.338539,1.0,5.25,9.5,13.75,18.0,18.0,421.388889,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,7.0,4.0,2.160247,1.0,2.50,4.0,5.50,7.0,7.0,368.285714,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1588,17.0,9.0,5.049752,1.0,5.00,9.0,13.00,17.0,17.0,519.352941,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1589,19.0,10.0,5.627314,1.0,5.50,10.0,14.50,19.0,19.0,388.315789,...,1.75,2.0,2.0,1.5,0.707107,1.0,1.25,1.5,1.75,2.0


In [5]:
# RandomForestClassifier can only track feature names of type string.
num_problems=0
for name in Xtrain.columns:
    if not isinstance(name,str):
        num_problems += 1
        print(type(name),name)
if num_problems==0:
    print("Ok")

Ok


In [6]:
print(datetime.datetime.now())
class RF_Util:
    def __init__(self):
        self.model=RandomForestClassifier()
    def get_model(self):
        return self.model
    def set_train(self,X,y):
        self.Xtr = X
        self.ytr = y
    def set_validation(self,X,y):
        self.Xval = X
        self.yval = y
    def fit(self):
        self.model=RandomForestClassifier()
        self.model.fit(self.Xtr,self.ytr)
    def cross_validation(self,fold=5):
        self.model=RandomForestClassifier()
        cv_scores = cross_val_score(self.model, self.Xtr, self.ytr, cv=fold)
        return cv_scores
    def validation_accuracy(self):
        ypred = self.model.predict(self.Xval)
        matches = np.count_nonzero(self.yval==ypred)
        accuracy = 100.0 * matches / len(ypred)  
        return accuracy
    def validation_confusion(self):
        ypred = self.model.predict(self.Xval)
        cm = confusion_matrix(self.yval, ypred)
        return cm
    def important_features(self):
        names = self.model.feature_names_in_
        importances = self.model.feature_importances_
        pairs = np.column_stack( (names,importances) )
        top_array = sorted(pairs, key = lambda e:e[1], reverse=True)
        # There must be a way to do this witout a loop!
        top_list = []
        for i in top_array:
             top_list.append((i[1],i[0]))  # 0=feature_name, 1=importance
        top_df = pd.DataFrame(top_list)
        return top_df

2022-06-06 16:49:54.557445


In [7]:
print(datetime.datetime.now())
print('Do a one-time train/valid split...')
Xt,Xv,yt,yv = train_test_split(Xtrain, ytrain.ravel()) 
        # ,random_state=42) # add this for reproducibility
print('Xtrain',Xt.shape,'ytrain',yt.shape,'non-zero:',np.count_nonzero(yt))
print('Xvalid',Xv.shape,'yvalid',yv.shape,'non-zero:',np.count_nonzero(yv))
print('Do a one-time fit and evaluate...')
rf1 = RF_Util()
rf1.set_train(Xt,yt)
rf1.set_validation(Xv,yv)
print(datetime.datetime.now())
rf1.fit()
print(datetime.datetime.now())
print('Accuracy:',rf1.validation_accuracy())
print('Confusion:')
print(rf1.validation_confusion())

2022-06-06 16:49:54.561382
Do a one-time train/valid split...
Xtrain (22640, 6392) ytrain (22640,) non-zero: 22640
Xvalid (7547, 6392) yvalid (7547,) non-zero: 7547
Do a one-time fit and evaluate...
2022-06-06 16:49:55.170835
2022-06-06 16:51:27.926680
Accuracy: 69.32555982509606
Confusion:
[[1954  494   59    8    0]
 [ 494 2192   72   16   24]
 [ 317  362  630    4    0]
 [ 135  170   37  273    2]
 [  45   65    8    3  183]]


In [8]:
Xt,Xv,yt,yv=None,None,None,None
rf1=None

In [9]:
print(datetime.datetime.now())
print("Shuffle...")
# Shuffle (happens automatically with cross validation but not with fit())
Xtrain,ytrain=shuffle(Xtrain,ytrain)  #set random_state for reproducibility

2022-06-06 16:51:28.602016
Shuffle...


In [10]:
print(datetime.datetime.now())
print('Cross-validation...')
rf2 = RF_Util()
rf2.set_train(Xtrain,ytrain)
cv_scores = rf2.cross_validation()
print(cv_scores)
print('mean %.4f +/- %.4f' % (cv_scores.mean(),cv_scores.std()))

2022-06-06 16:51:29.079463
Cross-validation...
[0.69095727 0.68797615 0.69786318 0.70117608 0.7038264 ]
mean 0.6964 +/- 0.0060


In [11]:
rf2=None

In [12]:
print(datetime.datetime.now())
print('Re-train on full training set...')
rf3 = RF_Util()
rf3.set_train(Xtrain,ytrain)
rf3.fit()
print(datetime.datetime.now())
print('...and rank the features by importance.')
top = rf3.important_features()

2022-06-06 16:59:53.907988
Re-train on full training set...
2022-06-06 17:02:02.773319
...and rank the features by importance.


In [15]:
pd.set_option('display.max_rows', None)
top.loc[:25]

,0,1
0,0.001641,Nuc_Texture_Correlation_Eosin_3_00_256_25%
1,0.001530,Nuc_Texture_Correlation_Eosin_3_00_256_mean
2,0.001456,Nuc_Texture_Correlation_Eosin_3_00_256_50%
3,0.001447,Nuc_Texture_SumAverage_Eosin_5_03_256_25%
4,0.001368,Nuc_Texture_SumAverage_Eosin_5_02_256_50%
5,0.001338,Nuc_Texture_SumAverage_Eosin_7_01_256_25%
6,0.001332,Nuc_Texture_InverseDifferenceMoment_Hematoxyli...
7,0.001263,Nuc_Texture_InverseDifferenceMoment_Hematoxyli...
8,0.001243,Nuc_Texture_SumAverage_Eosin_3_01_256_mean
9,0.001207,Nuc_Texture_SumAverage_Eosin_5_00_256_50%


In [14]:
print(datetime.datetime.now())

2022-06-06 17:02:02.821096
